# Overall playground to see how to connect AutoDraft to labelme

## First we need to dive into the JSON file that labelme creates in order to understand how to create one that it can then operate on

In [3]:
# imports
import numpy as np
import json
import pandas as pd
import base64
import io
from pprint import pprint
import PIL.Image
from labelme import utils
import os.path as osp
from labelme import PY2
from labelme import QT4
import base64
from labelme import __version__
#from IPython.core.debugger import set_trace

#auto-reload of python modules
%load_ext autoreload
%autoreload 2

### First lets figure out how to use modules

As long as the module is located within the same notebook directory it is simple to import directly. Ensure to import without the .py ending otherwise it will not import as a module.

In [4]:
import my_module

I am being executed!


In [5]:
my_module.square(4)

16

In [6]:
help(my_module)

Help on module my_module:

NAME
    my_module

DESCRIPTION
    Our first Python module. This initial string is a module-level documentation string.
    It is not a necessary component of the module. It is a useful way to describe the
    purpose of your module.

FUNCTIONS
    cube(x)
    
    square(x)

DATA
    some_list = ['a', 1, None]

FILE
    /Users/blank/AutoDraft/imagery/labelme_playground/my_module.py




### Viewing JSON data
JSON data when loaded into python prints extra ugly. The following allows for much more readable viewing of JSON files

In [7]:
data = json.load(open("ship1.json"))
# print(data) ## the data prints in a difficult string fashion
# pprint(data) ## one very quick way to print out the data. Looks okay...

In [9]:
pretty_data = json.dumps(data, ensure_ascii=False, indent=4) # prints JSON beautifully with indentations
#print(pretty_data)

### Now we need to figure out how to view/save image data

First method will be to attmept to use code within labelme

In [10]:
# Function provided in labelme source code
def load_image_file(filename):
    try:
        image_pil = PIL.Image.open(filename)
    except IOError:
        logger.error("Failed opening image file: {}".format(filename))
        return

    # apply orientation to image according to exif
    image_pil = utils.apply_exif_orientation(image_pil)

    with io.BytesIO() as f:
        ext = osp.splitext(filename)[1].lower()
        if PY2 and QT4:
            format = "PNG"
        elif ext in [".jpg", ".jpeg"]:
            format = "JPEG"
        else:
            format = "PNG"
        image_pil.save(f, format=format)
        f.seek(0)
        return f.read()

Now we can see how this actually works....

In [11]:
data = load_image_file("ship1.jpg")
#print(data)

Now we convert image data to base64, and then decode it to utf-8 for reading

In [12]:
new_data = base64.b64encode(data).decode("utf-8")
#print(new_data)

Great success!!! The above matches the data found in the applicable json file.
## Writing JSON files
Now we need to figure out how to write a simple JSON file, and then create one from scratch that labelme will open...the goal is just to write one that will match the current JSON file and that labelme will then open it successfully!

In [13]:
class LabelFile:
    
    #initializes the class
    def __init__(self, filename=None):
        self.shapes = []
        self.imagePath = None
        self.imageData = None
        if filename is not None:
            self.load(filename)
        self.filename = filename
    
    def load(self, filename):
        image_data = load_image_file(filename)
        imagePath = osp.dirname(filename)+filename
        flags = {} #this portion would need updating based on flags we want to add
        shapes = [
                dict(
                    label="waterline", # we would fill this with the different label strings
                    points=[[209,77],[228,81],[228,99],[210,99],[208,88]], # x-y point of shape that is being drawn
                    shape_type="polygon", #we can dictate more shapes here but this seems to be the most robust
                    flags={}, #flags that we create
                    group_id=None, # used for gouping similar items
                    other_data=None, # not really sure.....
                ) 
                #for s in data["shapes"]
            ]
        self.shapes = shapes
        self.imagePath = imagePath
        self.imageData = image_data
        self.flags = flags
        
        
    
    #this part checks the height and width of image and stores it
    @staticmethod
    def _check_image_height_and_width(imageData):
        img_arr = utils.img_b64_to_arr(imageData)
        imageHeight = img_arr.shape[0]
        imageWidth = img_arr.shape[1]
        return imageHeight, imageWidth
    
    #this portion saves and writes the JSON file
    def save(
        self,
        filename,
        shapes,
        imagePath,
        imageData,
        imageHeight=None,
        imageWidth=None,
        otherData=None,
        flags=None,
    ):
        if imageData is not None:
            imageData = base64.b64encode(imageData).decode("utf-8")
            imageHeight, imageWidth = self._check_image_height_and_width(imageData)
        if otherData is None:
            otherData = {}
        if flags is None:
            flags = {}
        data = dict(
            version=__version__,
            flags=flags,
            shapes=shapes,
            imagePath=imagePath,
            imageData=imageData,
            imageHeight=imageHeight,
            imageWidth=imageWidth,
        )
        try:
            if filename.endswith(".jpg"):
                filename = filename[:-4]
            with open(filename+".json", "w") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            #self.filename = filename
        except Exception as e:
            raise LabelFileError(e)
        

Alright now time to see what this produces

In [14]:
init = LabelFile("ship1_copy.jpg")

In [15]:
init.save(init.filename, init.shapes, init.imagePath, init.imageData)

## Hell yeah this totally worked!!!

In [16]:
data = json.load(open("ship1_copy.json"))
pretty_data = json.dumps(data, ensure_ascii=False, indent=4) # prints JSON beautifully with indentations
#print(pretty_data)